# Task 1: Iris Species Classification using Scikit-learn

## Objective
Build a decision tree classifier to predict iris species using the famous Iris dataset.

## Goals
1. Preprocess the data (handle missing values, encode labels)
2. Train a decision tree classifier
3. Evaluate using accuracy, precision, and recall
4. Visualize the decision tree and results


## Step 1: Import Required Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


## Step 2: Load and Explore the Dataset


In [ ]:
# Load the Iris dataset
iris = load_iris()

# Create a DataFrame for easier manipulation
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['species'] = iris.target
df['species_name'] = iris.target_names[iris.target]

print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nMissing Values:")
print(df.isnull().sum())

print("\nSpecies Distribution:")
print(df['species_name'].value_counts())


## Step 3: Data Visualization and Analysis


In [ ]:
# Create a comprehensive visualization of the dataset
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Pairplot for feature relationships
sns.scatterplot(data=df, x='sepal length (cm)', y='sepal width (cm)', 
                hue='species_name', ax=axes[0,0])
axes[0,0].set_title('Sepal Length vs Sepal Width')

sns.scatterplot(data=df, x='petal length (cm)', y='petal width (cm)', 
                hue='species_name', ax=axes[0,1])
axes[0,1].set_title('Petal Length vs Petal Width')

# 2. Box plots for feature distributions
df_melted = df.melt(id_vars=['species_name'], 
                    value_vars=iris.feature_names,
                    var_name='feature', value_name='value')

sns.boxplot(data=df_melted, x='feature', y='value', hue='species_name', ax=axes[1,0])
axes[1,0].set_title('Feature Distributions by Species')
axes[1,0].tick_params(axis='x', rotation=45)

# 3. Correlation heatmap
correlation_matrix = df[iris.feature_names].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[1,1])
axes[1,1].set_title('Feature Correlation Matrix')

plt.tight_layout()
plt.show()

# Print basic statistics
print("\nDataset Statistics:")
print(df[iris.feature_names].describe())


## Step 4: Data Preprocessing and Train-Test Split


In [ ]:
# Separate features and target
X = df[iris.feature_names]  # Features
y = df['species']  # Target (already encoded as 0, 1, 2)

print("Features shape:", X.shape)
print("Target shape:", y.shape)
print("\nTarget classes:", np.unique(y))
print("Target class names:", iris.target_names)

# Check for missing values (there shouldn't be any in Iris dataset)
print("\nMissing values in features:", X.isnull().sum().sum())
print("Missing values in target:", pd.Series(y).isnull().sum())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("\nTraining set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])
print("\nTraining set class distribution:")
print(pd.Series(y_train).value_counts().sort_index())
print("\nTest set class distribution:")
print(pd.Series(y_test).value_counts().sort_index())


## Step 5: Train Decision Tree Classifier


In [ ]:
# Create and train the decision tree classifier
dt_classifier = DecisionTreeClassifier(
    random_state=42,
    max_depth=3,  # Limit depth to prevent overfitting
    min_samples_split=5,  # Minimum samples to split a node
    min_samples_leaf=2   # Minimum samples in a leaf node
)

# Train the model
dt_classifier.fit(X_train, y_train)

print("Decision Tree Classifier trained successfully!")
print(f"Number of features: {dt_classifier.n_features_in_}")
print(f"Number of classes: {dt_classifier.n_classes_}")
print(f"Tree depth: {dt_classifier.get_depth()}")
print(f"Number of leaves: {dt_classifier.get_n_leaves()}")


## Step 6: Evaluate Model Performance


In [ ]:
# Make predictions
y_train_pred = dt_classifier.predict(X_train)
y_test_pred = dt_classifier.predict(X_test)

# Calculate metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("=== MODEL PERFORMANCE ===")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Calculate precision and recall for each class
precision = precision_score(y_test, y_test_pred, average=None)
recall = recall_score(y_test, y_test_pred, average=None)

print("\n=== DETAILED METRICS ===")
print("\nPer-class Precision:")
for i, species in enumerate(iris.target_names):
    print(f"{species}: {precision[i]:.4f}")

print("\nPer-class Recall:")
for i, species in enumerate(iris.target_names):
    print(f"{species}: {recall[i]:.4f}")

# Overall precision and recall (macro average)
macro_precision = precision_score(y_test, y_test_pred, average='macro')
macro_recall = recall_score(y_test, y_test_pred, average='macro')

print(f"\nMacro-averaged Precision: {macro_precision:.4f}")
print(f"Macro-averaged Recall: {macro_recall:.4f}")

# Classification report
print("\n=== CLASSIFICATION REPORT ===")
print(classification_report(y_test, y_test_pred, target_names=iris.target_names))


## Step 7: Visualize Results


In [ ]:
# Create confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=iris.target_names, 
            yticklabels=iris.target_names)
plt.title('Confusion Matrix - Decision Tree Classifier')
plt.xlabel('Predicted Species')
plt.ylabel('Actual Species')
plt.show()

# Visualize the decision tree
plt.figure(figsize=(20, 10))
plot_tree(dt_classifier, 
          feature_names=iris.feature_names,
          class_names=iris.target_names,
          filled=True,
          rounded=True,
          fontsize=10)
plt.title('Decision Tree Visualization', fontsize=16)
plt.show()

# Print feature importance
feature_importance = dt_classifier.feature_importances_
print("\n=== FEATURE IMPORTANCE ===")
for i, (feature, importance) in enumerate(zip(iris.feature_names, feature_importance)):
    print(f"{feature}: {importance:.4f}")

# Visualize feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importance, y=iris.feature_names, palette='viridis')
plt.title('Feature Importance in Decision Tree')
plt.xlabel('Importance Score')
plt.show()


## Step 8: Summary and Conclusions


In [ ]:
print("=== FINAL SUMMARY ===")
print(f"\n🎯 Model Performance:")
print(f"   • Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"   • Macro Precision: {macro_precision:.4f}")
print(f"   • Macro Recall: {macro_recall:.4f}")

print(f"\n🌳 Decision Tree Characteristics:")
print(f"   • Depth: {dt_classifier.get_depth()}")
print(f"   • Leaves: {dt_classifier.get_n_leaves()}")
print(f"   • Most important feature: {iris.feature_names[np.argmax(feature_importance)]}")

print(f"\n📊 Key Insights:")
print(f"   • The model achieved excellent performance on the Iris dataset")
print(f"   • Petal measurements are more important than sepal measurements")
print(f"   • The decision tree is interpretable and makes logical decisions")
print(f"   • No overfitting detected (small gap between train/test accuracy)")

print(f"\n✅ Task 1 Complete: Iris Classification with Decision Tree")
print(f"   All objectives achieved:")
print(f"   ✓ Data preprocessing completed")
print(f"   ✓ Decision tree classifier trained")
print(f"   ✓ Model evaluated with accuracy, precision, and recall")
print(f"   ✓ Results visualized and interpreted")
